<a href="https://colab.research.google.com/github/vivianconrad/neural-networks-and-deep-learning/blob/main/Inception_Score.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Inception Score using Keras - CIFAR10 dataset

In [ ]:
# calculating inception score for cifar-10 in Keras
from math import floor
from numpy import expand_dims
from numpy import log
from numpy import mean
from numpy import std
from numpy import exp
from numpy.random import shuffle
from keras.applications.inception_v3 import InceptionV3
from keras.applications.inception_v3 import preprocess_input
from keras.datasets import cifar10
from skimage.transform import resize
from numpy import asarray

In [ ]:
# scaling an array of images to a new size
def scale_images(images, new_shape):
	images_list = list()
 
	for image in images:
		# resizing with nearest neighbor interpolation
		new_image = resize(image, new_shape, 0)
		# store
		images_list.append(new_image)
	return asarray(images_list)

In [ ]:
# assuming images having any shape and pixels in [0,255]
def calculate_incp_score(images, n_split=10, eps=1E-16):

	# loading inception v3 model
	model = InceptionV3()
 
	# enumerating splits of images/predictions
	scores = list()
	n_part = floor(images.shape[0] / n_split)
 
	for i in range(n_split):
		# retrieving images
		ix_start, ix_end = i * n_part, (i+1) * n_part
		subset = images[ix_start:ix_end]

		# converting from uint8 to float32
		subset = subset.astype('float32')
  
		# scaling images to the required size
		subset = scale_images(subset, (299,299,3))
  
		# pre-processing images, scale to [-1,1]
		subset = preprocess_input(subset)
  
		# predicting p(y|x)
		p_yx = model.predict(subset)
  
		# calculating p(y)
		p_y = expand_dims(p_yx.mean(axis=0), 0)
  
		# calculating KL divergence using log probabilities
		kl_d = p_yx * (log(p_yx + eps) - log(p_y + eps))
  
		# summing over classes
		sum_kl_d = kl_d.sum(axis=1)
  
		# averaging over images
		avg_kl_d = mean(sum_kl_d)
  
		# undo the log
		incp_score = exp(avg_kl_d)
  
		# store
		scores.append(incp_score)
  
	# averaging across images
	is_avg, is_std = mean(scores), std(scores)
	return is_avg, is_std

In [ ]:
# load cifar10 images
(images, _), (_, _) = cifar10.load_data()

# shuffle images
shuffle(images)
images = images[:5000]
print('loaded', images.shape)

# calculate inception score
is_avg, is_std = calculate_incp_score(images)
print('score', is_avg, is_std)

loaded (5000, 32, 32, 3)
score 10.724917 0.22169621
